<a href="https://colab.research.google.com/github/dquinonez25/Python-Web-Scraping/blob/main/lessons/02_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web Scraping with Beautiful Soup

* * *

### Icons used in this notebook
🔔 **Question**: A quick question to help you understand what's going on.<br>
🥊 **Challenge**: Interactive exercise. We'll work through these in the workshop!<br>
⚠️ **Warning**: Heads-up about tricky stuff or common mistakes.<br>
💡 **Tip**: How to do something a bit more efficiently or effectively.<br>
🎬 **Demo**: Showing off something more advanced – so you know what Python can be used for!<br>

### Learning Objectives
1. [Reflection: To Scape Or Not To Scrape](#when)
2. [Extracting and Parsing HTML](#extract)
3. [Scraping the Illinois General Assembly](#scrape)

<a id='when'></a>

# To Scrape Or Not To Scrape

When we'd like to access data from the web, we first have to make sure if the website we are interested in offers a Web API. Platforms like Twitter, Reddit, and the New York Times offer APIs. **Check out D-Lab's [Python Web APIs](https://github.com/dlab-berkeley/Python-Web-APIs) workshop if you want to learn how to use APIs.**

However, there are often cases when a Web API does not exist. In these cases, we may have to resort to web scraping, where we extract the underlying HTML from a web page, and directly obtain the information we want. There are several packages in Python we can use to accomplish these tasks. We'll focus two packages: Requests and Beautiful Soup.

Our case study will be scraping information on the [state senators of Illinois](http://www.ilga.gov/senate), as well as the [list of bills](http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True) each senator has sponsored. Before we get started, peruse these websites to take a look at their structure.

## Installation

We will use two main packages: [Requests](http://docs.python-requests.org/en/latest/user/quickstart/) and [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/). Go ahead and install these packages, if you haven't already:

In [ ]:
%pip install requests

***Con*** *install requests* ***se realizan peticiones directamente desde el código.***

In [ ]:
%pip install beautifulsoup4

***Con*** *install beatufulsoup4* ***se extrae información de páginas web.***

We'll also install the `lxml` package, which helps support some of the parsing that Beautiful Soup performs:

In [ ]:
%pip install lxml

***Con*** *install lxml* ***se procesa y analiza documentos HTML y XML de forma rápida y eficiente.***

In [ ]:
# Import required libraries
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import time

***Se efectuan importación de librerías para poder realizar web scraping y extraer información de páginas web. Permite trabajar con fechas y horas, peticiones http, funciones para medir y controlar el tiempo, como*** *sleep()* ***para pausar la ejecución por unos segundos y evitar que el scraper haga demasiadas peticiones seguidas y sea bloqueado.***



<a id='extract'></a>

# Extracting and Parsing HTML

In order to succesfully scrape and analyse HTML, we'll be going through the following 4 steps:
1. Make a GET request
2. Parse the page with Beautiful Soup
3. Search for HTML elements
4. Get attributes and text of these elements

## Step 1: Make a GET Request to Obtain a Page's HTML

We can use the Requests library to:

1. Make a GET request to the page, and
2. Read in the webpage's HTML code.

The process of making a request and obtaining a result resembles that of the Web API workflow. Now, however, we're making a request directly to the website, and we're going to have to parse the HTML ourselves. This is in contrast to being provided data organized into a more straightforward `JSON` or `XML` output.

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp')
# Read the content of the server’s response
src = req.text
# View some output
print(src[:1000])

<!DOCTYPE html>
<html lang="en">
<head id="Head1">
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta http-equiv="content-type" content="text/html;charset=utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
    <meta charset="utf-8" />
    <meta charset="UTF-8">
    <!-- Meta Description -->
    <meta name="description" content="Welcome to the official government website of the Illinois General Assembly">
    <meta name="contactName" content="Legislative Information System">
    <meta name="contactOrganization" content="LIS Staff Services">
    <meta name="contactStreetAddress1" content="705 Stratton Office Building">
    <meta name="contactCity" content="Springfield">
    <meta name="contactZipcode" content="62706">
    <meta name="contactNetworkAddress" content="webmaster@ilga.gov">
    <meta name="contactPhoneNumber" content="217-782-3944">
    <meta name="contactFaxNumber" content="217-524-6059">
    <meta name


***Se ejecuta conexión a la página web del Senado de Illinois y se recupera el contenido de esa página. Se guarda el HTML completo de la respuesta en la variable*** src ***y se imprime los primeros 1000 caracteres.***


## Step 2: Parse the Page with Beautiful Soup

Now, we use the `BeautifulSoup` function to parse the reponse into an HTML tree. This returns an object (called a **soup object**) which contains all of the HTML in the original document.

If you run into an error about a parser library, make sure you've installed the `lxml` package to provide Beautiful Soup with the necessary parsing tools.

In [ ]:
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')
# Take a look
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html lang="en">
 <head id="Head1">
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta charset="utf-8"/>
  <!-- Meta Description -->
  <meta content="Welcome to the official government website of the Illinois General Assembly" name="description"/>
  <meta content="Legislative Information System" name="contactName"/>
  <meta content="LIS Staff Services" name="contactOrganization"/>
  <meta content="705 Stratton Office Building" name="contactStreetAddress1"/>
  <meta content="Springfield" name="contactCity"/>
  <meta content="62706" name="contactZipcode"/>
  <meta content="webmaster@ilga.gov" name="contactNetworkAddress"/>
  <meta content="217-782-3944" name="contactPhoneNumber"/>
  <meta content="217-524-6059" name="contactFaxNumber"/>
  <meta content="State Of Illinois" name="originatorJur

***Se convierte el contenido HTML de la página web, y el que se obtuvo con requests se lo convierte en un árbol de elementos HTML. Con parser*** lxml ***se analiza HTML y XML. Devuelve el HTML que permite embellecer facilitando leer la estructura del documento y muestra los primeros 1000 caracteres para no saturar la salida.***

The output looks pretty similar to the above, but now it's organized in a `soup` object which allows us to more easily traverse the page.

## Step 3: Search for HTML Elements

Beautiful Soup has a number of functions to find useful components on a page. Beautiful Soup lets you find elements by their:

1. HTML tags
2. HTML Attributes
3. CSS Selectors

Let's search first for **HTML tags**.

The function `find_all` searches the `soup` tree to find all the elements with an a particular HTML tag, and returns all of those elements.

What does the example below do?

In [ ]:
# Find all elements with a certain tag
a_tags = soup.find_all("a")
print(a_tags[:10])

[<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="af" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-za"></span> Afrikaans
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="sq" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-al"></span> Albanian
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="ar" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-ae"></span> Arabic
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="hy" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-am"></span> Armenian
                            </a>, <a b-0yw6sxot5c="" class="dropdown-item" data-lang="az" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-az"></span> Azerbaijani
            

***Busca todas las etiquetas*** < a > ***en el documento HTML, toma los primeros 10 elementos encontrados y los muestra en la consola.***

Because `find_all()` is the most popular method in the Beautiful Soup search API, you can use a shortcut for it. If you treat the BeautifulSoup object as though it were a function, then it’s the same as calling `find_all()` on that object.

These two lines of code are equivalent:

In [ ]:
a_tags = soup.find_all("a")
a_tags_alt = soup("a")

if a_tags:
    print(a_tags[0])
else:
    print("No 'a' tags found.")

if a_tags_alt:
    print(a_tags_alt[0])
else:
    print("No 'a' tags found using the alternative method.")

No 'a' tags found.
No 'a' tags found using the alternative method.


***Busca todas las etiquetas*** < a > ***en el documento HTML y devuelve una lista con todos los elementos encontrados. Verifica si se encontraron enlaces y muestra el primero, caso contrario imprime un mensaje indicando que ninguno se encontró y replica el proceso con la variable alternativa*** a_tags_alt

How many links did we obtain?

In [ ]:
print(len(a_tags))

0


***Muestra cuántas etiquetas*** < a > ***hay en la página web analizada con BeautifulSoup.***

That's a lot! Many elements on a page will have the same HTML tag. For instance, if you search for everything with the `a` tag, you're likely to get more hits, many of which you might not want. Remember, the `a` tag defines a hyperlink, so you'll usually find many on any given page.

What if we wanted to search for HTML tags with certain attributes, such as particular CSS classes?

We can do this by adding an additional argument to the `find_all`. In the example below, we are finding all the `a` tags, and then filtering those with `class_="sidemenu"`.

In [ ]:
# Get only the 'a' tags in 'sidemenu' class
side_menus = soup("a", class_="sidemenu")
side_menus[:5]

[]

***Analiza el documento HTML y extrae enlaces específicos, busca todas las etiquetas*** < a > ***que tengan la clase*** *sidemenu* ***y toma los primeros 5 elementos encontrados.***

A more efficient way to search for elements on a website is via a **CSS selector**. For this we have to use a different method called `select()`. Just pass a string into the `.select()` to get all elements with that string as a valid CSS selector.

In the example above, we can use `"a.sidemenu"` as a CSS selector, which returns all `a` tags with class `sidemenu`.

In [ ]:
# Get elements with "a.sidemenu" CSS Selector.
selected = soup.select("a.sidemenu")
selected[:5]

[]

***Usando un selector CSS encuentra todas las etiquetas*** < a > ***que tengan la clase*** sidemenu ***y selecciona los primeros 5 elementos encontrados.***

## 🥊 Challenge: Find All

Use BeautifulSoup to find all the `a` elements with class `mainmenu`.

In [ ]:
# YOUR CODE HERE


## Step 4: Get Attributes and Text of Elements

Once we identify elements, we want the access information in that element. Usually, this means two things:

1. Text
2. Attributes

Getting the text inside an element is easy. All we have to do is use the `text` member of a `tag` object:

In [ ]:
# Get all sidemenu links as a list
# Make a GET request
import requests
from bs4 import BeautifulSoup

req = requests.get('http://www.ilga.gov/senate/default.asp')
# Read the content of the server’s response
src = req.text
# Parse the response into an HTML tree
soup = BeautifulSoup(src, 'lxml')

# Select all 'a' tags instead of those with class 'sidemenu'
all_links = soup.select("a")

# Examine the first link, checking if the list is not empty
if all_links:
    first_link = all_links[0]
    print(first_link)

    # What class is this variable?
    print('Class: ', type(first_link))
else:
    print("No links found on the page.")

<a b-0yw6sxot5c="" class="dropdown-item" data-lang="en" href="#">
<span b-0yw6sxot5c="" class="flag-icon flag-icon-us"></span> English
                            </a>
Class:  <class 'bs4.element.Tag'>


***Busca todos los elementos*** < a > ***que tengan la clase*** sidemenu ***que se ha cargado en la variable*** soup ***y devuelve una lista con todos esos elementos. Toma el primer elemento de esa lista y muestra en pantalla el HTML completo de ese primer enlace.***

It's a Beautiful Soup tag! This means it has a `text` member:

In [ ]:
# Check if first_link is defined before printing its text
if 'first_link' in locals():
    print(first_link.text)
else:
    print("first_link is not defined.")


 English
                            


***Devuelve un diccionario con todas las variables locales definidas en ese momento y comprueba si la variable*** first_link ***está entre ellas. Si está definida se imprime el texto dentro del enlace, caso contrario imprime un mensaje de notificación.***

Sometimes we want the value of certain attributes. This is particularly relevant for `a` tags, or links, where the `href` attribute tells us where the link goes.

💡 **Tip**: You can access a tag’s attributes by treating the tag like a dictionary:

In [ ]:
# Check if first_link is defined before accessing its attributes
if 'first_link' in locals():
    print(first_link['href'])
else:
    print("first_link is not defined.")

#


***Devuelve un diccionario con todas las variables locales definidas en ese momento y comprueba si la variable*** *first_link* ***existe. Si existe imprime el valor del atributo*** *href* ***que establece el destino de un enlace, caso contrario imprime un mensaje que no está definido.***

## 🥊 Challenge: Extract specific attributes

Extract all `href` attributes for each `mainmenu` URL.

In [ ]:
# YOUR CODE HERE


<a id='scrape'></a>

# Scraping the Illinois General Assembly

Believe it or not, those are really the fundamental tools you need to scrape a website. Once you spend more time familiarizing yourself with HTML and CSS, then it's simply a matter of understanding the structure of a particular website and intelligently applying the tools of Beautiful Soup and Python.

Let's apply these skills to scrape the [Illinois 98th General Assembly](http://www.ilga.gov/senate/default.asp?GA=98).

Specifically, our goal is to scrape information on each senator, including their name, district, and party.

## Scrape and Soup the Webpage

Let's scrape and parse the webpage, using the tools we learned in the previous section.

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")

***Envía una solicitud HTTP GET a la URL especificada y recupera el contenido de la página. Guarda el contenido HTML de la respuesta en la variable*** src ***y dicho código fuente como texto plano. Luego convierte el texto HTML en un objeto para recorrerlo e interpretarlo.***

## Search for the Table Elements

Our goal is to obtain the elements in the table on the webpage. Remember: rows are identified by the `tr` tag. Let's use `find_all` to obtain these elements.

In [ ]:
# Get all table row elements
rows = soup.find_all("tr")
len(rows)

0

***Busca todos los elementos*** *tr*, ***es decir una fila de tabla y devuelve el número total de filas encontradas.***

⚠️ **Warning**: Keep in mind: `find_all` gets *all* the elements with the `tr` tag. We only want some of them. If we use the 'Inspect' function in Google Chrome and look carefully, then we can use some CSS selectors to get just the rows we're interested in. Specifically, we want the inner rows of the table:

In [ ]:
# Filter rows to find the relevant ones.
# Based on the HTML structure, the relevant rows seem to contain 'td' elements with the class 'detail'.
# Assuming the rows are within a table, we can select 'tr' within the table body or a specific table id/class.
# Let's refine the row selection based on the structure observed in the output of cell 0_esi0UGsbHH
# We can look for rows that contain 'td' elements with the class 'detail' within a table.
# A more robust approach might be to look for rows within the main content area or a specific table.
# Based on the HTML structure, the relevant rows seem to be within a table. Let's try selecting 'tr' elements
# that are descendants of a table. We'll still filter based on the presence of 'td.detail'.

# Re-fetch the soup object as it might have been modified or cleared in previous steps
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
src = req.text
soup = BeautifulSoup(src, "lxml")

# Select all table rows. We will filter later.
all_table_rows = soup.find_all("tr")

# Filter rows to find the relevant ones. We are looking for rows that contain 'td' elements with the class 'detail'.
# This assumes that the senator information is within such td elements.
rows = [row for row in all_table_rows if row.select('td.detail')]


for row in rows[:5]:
    print(row, '\n')

***Recorre cada lista*** *row*, ***verifica si contiene al menos un elemento*** *td* ***con clase*** *detail*, ***conserva aquellas filas que si tienen, imprime los primeros cinco elementos filtrados y se agrega una línea en blanco entre cada impresión para facilitar la lectura.***

It looks like we want everything after the first two rows. Let's work with a single row to start, and build our loop from there.

In [ ]:
# Check if the rows list is not empty before accessing elements
if rows:
    example_row = rows[0] # Use the first valid row as an example
    print(example_row.prettify())
else:
    print("No relevant rows found on the page.")

No relevant rows found on the page.


***Verifica si la lista*** *rows* ***no está vacía, si hay elementos toma el primero de la lista e imprime el contenido de esa fila. Caso contrario si la lista está vacía, imprime un mensaje indicando que no se encontraron filas relevantes.***

Let's break this row down into its component cells/columns using the `select` method with CSS selectors. Looking closely at the HTML, there are a couple of ways we could do this.

* We could identify the cells by their tag `td`.
* We could use the the class name `.detail`.
* We could combine both and use the selector `td.detail`.

In [ ]:
# Check if the rows list is not empty before accessing elements and then select the first row
if rows:
    example_row = rows[0]
    for cell in example_row.select('td'):
        print(cell)
    print()

    for cell in example_row.select('.detail'):
        print(cell)
    print()

    for cell in example_row.select('td.detail'):
        print(cell)
    print()
else:
    print("No rows found to process.")

No rows found to process.


***Verifica si la lista*** *rows* ***contiene elementos, selecciona la primera fila como ejemplo para inspección, muestra cada celda de la fila, extrae cualquier elemento con clase*** *detail*, ***se busca solo celdas*** < td > ***que tengan la clase*** *detail*, ***caso contrario imprime un mensaje que no hay filas disponibles para analizar.***

We can confirm that these are all the same.

In [ ]:
# Check if example_row is defined before asserting
if 'example_row' in locals():
    assert example_row.select('td') == example_row.select('.detail') == example_row.select('td.detail')
else:
    print("example_row is not defined, cannot perform assertion.")

example_row is not defined, cannot perform assertion.


***Valida si la variable está definida y existe en el entorno local, compara tres selecciones de elementos y comprueba si los tres resultados son iguales el programa continúa, caso contrario imprime que no está definido, no se puede realizar la afirmación.***

Let's use the selector `td.detail` to be as specific as possible.

In [ ]:
# Select only those 'td' tags with class 'detail'
if 'example_row' in locals():
    detail_cells = example_row.select('td.detail')
    detail_cells
else:
    print("example_row is not defined. Please ensure relevant rows are found before proceeding.")

example_row is not defined. Please ensure relevant rows are found before proceeding.


***Verifica si la variable está definida, busca todas las celdas*** < td > ***dentro de esa fila que tengan la clase detail, guarda en la variable*** *detail_cells*, ***caso contrario imprime que la variable no existe en el entorno local y que no se encontraron filas relevantes.***

Most of the time, we're interested in the actual **text** of a website, not its tags. Recall that to get the text of an HTML element, we use the `text` member:

In [ ]:
# Keep only the text in each of those cells
if 'detail_cells' in locals():
    row_data = [cell.text for cell in detail_cells]
    print(row_data)
else:
    print("detail_cells is not defined. Please ensure example_row and detail_cells are properly defined before proceeding.")

detail_cells is not defined. Please ensure example_row and detail_cells are properly defined before proceeding.


***Verifica si la variable*** *detail_cells existe* ***en el entorno local, recorre cada elemento y extrae solo el texto visible dentro de cada celda. Muestra la lista de textos extraídos, caso contrario imprime un mensaje de advertencia para que sea revisado antes de continuar.***

Looks good! Now we just use our basic Python knowledge to get the elements of this list that we want. Remember, we want the senator's name, their district, and their party.

In [ ]:
print(row_data[0]) # Name
print(row_data[3]) # District
print(row_data[4]) # Party

row_data is not defined. Please ensure detail_cells and row_data are properly defined before proceeding.


***Verifica si la variable existe en el entorno local. Muestra el primer, cuarto y quinto elemento de la lista con su respectivo nombre; caso contrario imprime un mensaje de advertencia para que sea revisado antes de continuar.***

## Getting Rid of Junk Rows

We saw at the beginning that not all of the rows we got actually correspond to a senator. We'll need to do some cleaning before we can proceed forward. Take a look at some examples:

In [ ]:
if rows:
    print('Row 0:\n', rows[0], '\n')
    print('Row 1:\n', rows[1], '\n')
    print('Last Row:\n', rows[-1])
else:
    print("The 'rows' list is empty. No rows were found on the page in the previous step.")

The 'rows' list is empty. No rows were found on the page in the previous step.


**Verifica si la lista** rows ***contiene elementos, muestra el contenido completo de la primera fila, muestra la segunda fila útil para comparar estructura o contenido, muestra la última fila de la lista usando índice negativo. Caso contrario imprime un mensaje indicando que no se encontraron filas en el paso anterior.***

When we write our for loop, we only want it to apply to the relevant rows. So we'll need to filter out the irrelevant rows. The way to do this is to compare some of these to the rows we do want, see how they differ, and then formulate that in a conditional.

As you can imagine, there a lot of possible ways to do this, and it'll depend on the website. We'll show some here to give you an idea of how to do this.

In [ ]:
if len(rows) >= 4:
    # Bad rows
    print(len(rows[0]))
    print(len(rows[1]))

    # Good rows
    print(len(rows[2]))
    print(len(rows[3]))
else:
    print("The 'rows' list does not contain enough elements to perform this check.")

The 'rows' list does not contain enough elements to perform this check.


***Verifica que haya al menos 4 filas en la lista*** rows, ***imprime la cantidad de elementos en las dos primeras filas, que se consideran malas o inútiles; y en las siguientes dos filas, imprime las que se consideran buenas o útiles para extraer datos. Caso contrario si hay menos de 4 filas, evita el análisis e informa que no hay suficientes datos.***

Perhaps good rows have a length of 5. Let's check:

In [ ]:
good_rows = [row for row in rows if len(row) == 5]

# Let's check some rows
if good_rows:
    print(good_rows[0], '\n')
    # Ensure there are at least 2 elements for [-2]
    if len(good_rows) >= 2:
        print(good_rows[-2], '\n')
    print(good_rows[-1])
else:
    print("The 'good_rows' list is empty. No rows with length 5 were found.")

The 'good_rows' list is empty. No rows with length 5 were found.


***Se valida creación de una nueva lista*** *good_rows* ***que solo incluye las filas que tienen exactamente 5 elementos, si hay filas útiles muestra, caso contrario imprime que no se encontraron filas válidas.***

We found a footer row in our list that we'd like to avoid. Let's try something else:

In [ ]:
if len(rows) > 2:
    rows[2].select('td.detail')
else:
    print("The 'rows' list does not contain enough elements to access index 2.")

The 'rows' list does not contain enough elements to access index 2.


***Valida que la lista*** *rows* ***tenga al menos tres elementos, los índices 0, 1 y 2. Accede a la tercera fila y selecciona solo las celdas*** < td > ***que tengan la clase*** *detail.* ***Caso contrario imprime un mensaje indicando que no hay suficientes filas y no se puede acceder al índice 2.***

In [ ]:
if len(rows) > 5:
    # Bad row
    print(rows[-1].select('td.detail'), '\n')

    # Good row
    print(rows[5].select('td.detail'), '\n')

# How about this?
good_rows = [row for row in rows if row.select('td.detail')]

if good_rows:
    print("Checking rows...\n")
    print(good_rows[0], '\n')
    print(good_rows[-1])
else:
    print("The 'good_rows' list is empty. No rows with 'td.detail' were found.")

The 'good_rows' list is empty. No rows with 'td.detail' were found.


***Valida que hay al menos 6 filas, imprime la última posiblemente sea mala o no deseada, imprime la sexta fila, que se considera buena o con datos válidos, se filtra las filas útiles limpiando la tabla y queda con lo que interesa. Si hay filas buenas muestra la primera y última, caso contrario imprime que no hay e informa que la lista está vacía.***

Looks like we found something that worked!

## Loop it All Together

Now that we've seen how to get the data we want from one row, as well as filter out the rows we don't want, let's put it all together into a loop.

In [ ]:
# Define storage list
members = []

# Get rid of junk rows
valid_rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in valid_rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]
    # Store in a tuple
    senator = (name, district, party)
    # Append to list
    members.append(senator)

***Guarda los datos de cada senador, filtra las filas útiles descartando encabezados, filas vacías o decorativas. Comienza un bucle para procesar cada fila que contiene datos útiles, extrae las celdas con clase*** *detail* ***y se obtiene sólo las relevantes dentro de la fila. Convierte las celdas HTML en texto plano, extrae campos específicos, agrupa los datos en una estructura simple y ordenada, y guarda la información en la lista*** *members.*

In [ ]:
# Should be 61
len(members)

0

***Devuelve el número total de tuplas almacenadas en la lista*** *members.*

Let's take a look at what we have in `members`.

In [ ]:
print(members[:5])

[]


***Imprime los primeros cinco elementos de la lista*** *members.*

## 🥊  Challenge: Get `href` elements pointing to members' bills

The code above retrieves information on:  

- the senator's name,
- their district number,
- and their party.

We now want to retrieve the URL for each senator's list of bills. Each URL will follow a specific format.

The format for the list of bills for a given senator is:

`http://www.ilga.gov/senate/SenatorBills.asp?GA=98&MemberID=[MEMBER_ID]&Primary=True`

to get something like:

`http://www.ilga.gov/senate/SenatorBills.asp?MemberID=1911&GA=98&Primary=True`

in which `MEMBER_ID=1911`.

You should be able to see that, unfortunately, `MEMBER_ID` is not currently something pulled out in our scraping code.

Your initial task is to modify the code above so that we also **retrieve the full URL which points to the corresponding page of primary-sponsored bills**, for each member, and return it along with their name, district, and party.

Tips:

* To do this, you will want to get the appropriate anchor element (`<a>`) in each legislator's row of the table. You can again use the `.select()` method on the `row` object in the loop to do this — similar to the command that finds all of the `td.detail` cells in the row. Remember that we only want the link to the legislator's bills, not the committees or the legislator's profile page.
* The anchor elements' HTML will look like `<a href="/senate/Senator.asp/...">Bills</a>`. The string in the `href` attribute contains the **relative** link we are after. You can access an attribute of a BeatifulSoup `Tag` object the same way you access a Python dictionary: `anchor['attributeName']`. See the <a href="http://www.crummy.com/software/BeautifulSoup/bs4/doc/#tag">documentation</a> for more details.
* There are a _lot_ of different ways to use BeautifulSoup to get things done. whatever you need to do to pull the `href` out is fine.

The code has been partially filled out for you. Fill it in where it says `#YOUR CODE HERE`. Save the path into an object called `full_path`.

In [ ]:
# Make a GET request
req = requests.get('http://www.ilga.gov/senate/default.asp?GA=98')
# Read the content of the server’s response
src = req.text
# Soup it
soup = BeautifulSoup(src, "lxml")
# Create empty list to store our data
members = []

# Returns every ‘tr tr tr’ css selector in the page
rows = soup.select('tr tr tr')
# Get rid of junk rows
rows = [row for row in rows if row.select('td.detail')]

# Loop through all rows
for row in rows:
    # Select only those 'td' tags with class 'detail'
    detail_cells = row.select('td.detail')
    # Keep only the text in each of those cells
    row_data = [cell.text for cell in detail_cells]
    # Collect information
    name = row_data[0]
    district = int(row_data[3])
    party = row_data[4]

    # YOUR CODE HERE
    full_path = ''

    # Store in a tuple
    senator = (name, district, party, full_path)
    # Append to list
    members.append(senator)

**Hace una petición GET al sitio web del Senado de Illinois para obtener la lista de senadores, extrae el contenido HTML como texto plano desde la respuesta, utiliza** *BeautifulSoup* ***con el parser*** lxml ***para convertir el HTML en una estructura que pueda navegar fácilmente, crea una lista vacía para guardar la información de cada senador, selecciona todos los elementos*** < tr > ***anidados tres veces y filtra las filas que contiene al menos una celda*** < td > ***con clase*** *detail.* ***Selecciona las celdas dentro de cada fila y extrae el texto de cada una. Organiza los datos, crea una tupla con esa información y la agrega a la lista*** *members.*

In [ ]:
# Uncomment to test
# members[:5]

## 🥊  Challenge: Modularize Your Code

Turn the code above into a function that accepts a URL, scrapes the URL for its senators, and returns a list of tuples containing information about each senator.

In [ ]:
# YOUR CODE HERE
def get_members(url):
    return [___]


***Se define*** *get_members* ***que recibe un parámetro url y que retorna una lista de miembros extraídos desde la misma.***

In [ ]:
# Test your code
url = 'http://www.ilga.gov/senate/default.asp?GA=98'
senate_members = get_members(url)
len(senate_members)

1

***Define la URL de la página web que contiene los datos del Senado de Illinois, hace web scraping a la página para obtener una lista de los senadores, el resultado se guarda en una variable*** *senate_members* ***y calcula la cantidad total de miembros que fueron extraídos de la página.***

## 🥊 Take-home Challenge: Writing a Scraper Function

We want to scrape the webpages corresponding to bills sponsored by each bills.

Write a function called `get_bills(url)` to parse a given bills URL. This will involve:

  - requesting the URL using the <a href="http://docs.python-requests.org/en/latest/">`requests`</a> library
  - using the features of the `BeautifulSoup` library to find all of the `<td>` elements with the class `billlist`
  - return a _list_ of tuples, each with:
      - description (2nd column)
      - chamber (S or H) (3rd column)
      - the last action (4th column)
      - the last action date (5th column)
      
This function has been partially completed. Fill in the rest.

In [ ]:
def get_bills(url):
    src = requests.get(url).text
    soup = BeautifulSoup(src)
    rows = soup.select('tr')
    bills = []
    for row in rows:
        # Find all td elements within the current row
        cells = row.select('td')
        # Check if the row has enough cells and the class is 'billlist'
        if len(cells) >= 5 and 'billlist' in cells[1].get('class', []):
            # YOUR CODE HERE
            bill_id = cells[0].text.strip() if len(cells) > 0 else ''
            description = cells[1].text.strip() if len(cells) > 1 else ''
            chamber = cells[2].text.strip() if len(cells) > 2 else ''
            last_action = cells[3].text.strip() if len(cells) > 3 else ''
            last_action_date = cells[4].text.strip() if len(cells) > 4 else ''

            bill = (bill_id, description, chamber, last_action, last_action_date)
            bills.append(bill)
    return bills

***Obtiene el contenido HTML de la página y lo descarga como texto desde la URL, lo convierte en un objeto BeautifulSoup para poder analizarlo, selecciona todas las filas*** < tr > ***de la tabla, itera sobre cada fila y busca las celdas*** < td >, ***filtra las filas relevantes procesando sólo las que tienen al menos 5 celdas y cuya segunda celda tiene la clase*** *billlist.* ***Extrae los datos de cada celda, guarda cada proyecto como una tupla, lo agrega a la lista bills y retorna la lista completa de proyectos.***

In [ ]:
# Uncomment to test your code
# test_url = senate_members[0][3]
# get_bills(test_url)[0:5]

### Scrape All Bills

Finally, create a dictionary `bills_dict` which maps a district number (the key) onto a list of bills (the value) coming from that district. You can do this by looping over all of the senate members in `members_dict` and calling `get_bills()` for each of their associated bill URLs.

**NOTE:** please call the function `time.sleep(1)` for each iteration of the loop, so that we don't destroy the state's web site.

In [ ]:
# YOUR CODE HERE


In [ ]:
# Uncomment to test your code
# bills_dict[52]